# **Installing tidyverse package and loading libraries**
Checking if a package is installed and if not installed it is installed. If installed it is loaded.  
"install.packages("tidyverse")" installs tidyverse

In [1]:
cat("\nChecking if needed packages are installed... 'tidyverse','dplyr' and 'magrittr'")
if("tidyverse" %in% rownames(installed.packages()) == FALSE) {
        install.packages("tidyverse")
} else {
        cat("\nExcellent tidyverse already installed.\nproceeding with clean_up and sorting\n")
	## loads dplyr and magrittr packages
	suppressMessages(library(dplyr))
	suppressMessages(library(magrittr))
	suppressMessages(library(tools))
}



Checking if needed packages are installed... 'tidyverse','dplyr' and 'magrittr'
Excellent tidyverse already installed.
proceeding with clean_up and sorting


## **Loading RDPclassifier results in tab delimited format to a dataframe object.** 

In [2]:
bold_file <- "/home/kibet/bioinformatics/github/co1_metaanalysis/data/input/bold_data/diptera_13012021/diptera_13012021.tsv"
#args = commandArgs(trailingOnly=TRUE)
filename_ext = file_ext(bold_file)
filename = file_path_sans_ext(bold_file)
input_src=dirname(file_path_as_absolute(bold_file))
bold_dataframe = read.delim(bold_file, stringsAsFactors = F, header = T, na.strings = "")
## loads bold2.tsv as a dataframe object. works ok, bold2.tsv does not contain any '\r' characters

In [3]:
str(bold_dataframe)

'data.frame':	81302 obs. of  80 variables:
 $ processid                 : chr  "GBDP32377-19" "GBDP32805-19" "GBDP32808-19" "GBDP32817-19" ...
 $ sampleid                  : chr  "MF446534" "KX853282" "KX853251" "KX853243" ...
 $ recordID                  : int  9905930 9906358 9906361 9906370 9906371 9906373 9906469 9906470 9906472 9906578 ...
 $ catalognum                : chr  NA NA NA NA ...
 $ fieldnum                  : chr  "ZFMK_D190" NA NA NA ...
 $ institution_storing       : chr  "Mined from GenBank, NCBI" "Mined from GenBank, NCBI" "Mined from GenBank, NCBI" "Mined from GenBank, NCBI" ...
 $ collection_code           : logi  NA NA NA NA NA NA ...
 $ bin_uri                   : chr  "BOLD:AAZ6524" NA NA "BOLD:AED2981" ...
 $ phylum_taxID              : int  20 20 20 20 20 20 20 20 20 20 ...
 $ phylum_name               : chr  "Arthropoda" "Arthropoda" "Arthropoda" "Arthropoda" ...
 $ class_taxID               : int  82 82 82 82 82 82 82 82 82 82 ...
 $ class_name            

## **Taking a closer look at the bold_dataframe$nucleotides field**
Introducing a field "seqlen1" that has the number of nucleotides in the COI_data

In [4]:
bold_data = subset(bold_dataframe, !is.na(nucleotides))
bold_data %>% mutate(seqlen1 = nchar(nucleotides)) -> resulting_dataframe1 

## **REMOVING '-'characters from nucleotide sequences and creating a field of unalinged nucleotide sequences**
(resulting_dataframe2$unaligned_nucleotides)


In [5]:
resulting_dataframe1 %>% mutate(unaligned_nucleotides = gsub('-', '', resulting_dataframe1$nucleotides, ignore.case = FALSE, perl = FALSE, fixed = FALSE, useBytes = FALSE)) -> resulting_dataframe2 

## **Introducing a field seqlen2 with number nucleotides in resulting_dataframe2$unaligned_nucleotides field**


In [6]:
resulting_dataframe2 %>% mutate(seqlen2 = nchar(unaligned_nucleotides)) -> resulting_dataframe3 

## **Taking a closer look at resulting_dataframe3$markercode**
Filtering records corresponding to COI-5P markers

In [8]:
COI_data = subset(resulting_dataframe3, 
                  markercode == "COI-5P" & !is.na(nucleotides))
non_COI_data = subset(resulting_dataframe3, 
                      markercode != "COI-5P" & !is.na(nucleotides))

## **Generating a file with all 'COI-5P' sequences**

In [9]:
COI_data -> COI_all_data
cat("\nOf all",nrow(bold_dataframe), "retrieved records, ",
    length(COI_all_data$unaligned_nucleotides), " sequences have 'COI-5P' marker.",
    " These are the only ones retained for downstream analysis::\n")


Of all 81302 retrieved records,  77892  sequences have 'COI-5P' marker.  These are the only ones retained for downstream analysis::


## **Introducing a filter to remove sequences with less than 500 nucleotides**

In [11]:
COI_all_data %>% filter(seqlen2 >= 500 ) -> COI_Over499_data
cat("\t",length(COI_Over499_data$unaligned_nucleotides),
    "sequences have 500 or more nucleotide bases\n")

	 32351 sequences have 500 or more nucleotide bases


## **Introducing a filter to remove any sequence with less than 500 and more than 700 nucleotides**


In [12]:
COI_all_data %>% filter(seqlen2 >= 500 & seqlen2 <= 700) -> COI_500to700_data
cat("\t",length(COI_500to700_data$unaligned_nucleotides),
    "sequences have 500 to 700 bases\n")

	 32302 sequences have 500 to 700 bases


## **Introducing a filter to remove any sequence with less than 650 and over 660 nucleotides**

In [13]:
COI_all_data %>% filter(seqlen2 >= 650 & seqlen2 <= 660) -> COI_650to660_data
cat("\t",length(COI_650to660_data$unaligned_nucleotides),
    "sequences have 650 to 660 bases\n") 

	 4415 sequences have 650 to 660 bases


## **Introducing a filter to remove any sequence with over 500 nucleotides**

In [14]:
COI_all_data %>% filter(seqlen2 < 500) -> COI_Under500_data
cat("\t",length(COI_Under500_data$unaligned_nucleotides),
    "sequences have less than 500 bases\n")

	 902 sequences have less than 500 bases


## **Introducing a filter to remove any sequence with less than 700 nucleotides**

In [15]:
COI_all_data %>% filter(seqlen2 > 700) -> COI_Over700_data
cat("\t",length(COI_Over700_data$unaligned_nucleotides),
    "sequences have more than 700 bases\n")

	 49 sequences have more than 700 bases


## **Assessing those with geographical coordinates**

In [19]:
cat( "\t", nrow(subset (COI_all_data, is.na(lat))), "records lack latitude co-ordinates \n\t",
    nrow(subset (COI_all_data, is.na(lon))), "records lack longitude co-ordinates")

	 1750 records lack latitude co-ordinates 
	 1750 records lack longitude co-ordinates

## **Summary**

In [37]:
cat( "\n", nrow(COI_data), "records have COI-5P marker sequences",
    nrow(subset(bold_dataframe, markercode != "COI-5P")),
    "have non-COI-5P markersequences",
    nrow(subset(bold_dataframe,  is.na(nucleotides) )),
    "lack nucleotide sequence values (is.na(nucleotides))")


 33253 records have COI-5P marker sequences 43 have non-COI-5P markersequences 185 lack nucleotide sequence values (is.na(nucleotides))

## **Printing copies of the final tidy files as dataframes in .tsv format**

In [ ]:
output_list = c("COI_all_data", "non_COI_data", "COI_Over499_data", "COI_500to700_data", "COI_650to660_data", "COI_Over700_data", "COI_Under500_data")
output_list_names = c("_all_data", "_NonCOI_data", "_Over499_data", "_500to700_data", "_650to660_data", "_Over700_data", "_Under500_data")

datalist = lapply(output_list, get)
names(datalist) <- paste(input_src,"/", filename, output_list_names, sep= "" )
for (i in 1:length(datalist)) {write.table(datalist[i], file = paste(names(datalist[i]), ".tsv", sep = ""), row.names = FALSE, col.names= TRUE, sep = "\t", quote=FALSE)}